## Project Final for ECE 556: AI for Radar and Remote Sensing
Paul Delgado

In [1]:
# Python packages
import csv
import os
import pathlib
import requests
import sys
import zipfile

In [2]:
# Third-party packages
import mne_bids
import openneuro
import tensorflow as tf

In [3]:
import os
import pickle
from datetime import datetime
from itertools import product

In [155]:
#import caffe
import numpy as np
import PIL.Image
import scipy.io as sio

In [171]:
sys.path.insert(1, os.path.normpath(os.path.join(os.getcwd(), '..', 'icnn')))
from icnn.icnn_dgn_gd import reconstruct_image
from icnn.utils import clip_extreme_value, normalise_img

ModuleNotFoundError: No module named 'caffe'

### Prepare Dataset

In [85]:
dataset = 'ds001506'
subject = '01'
session = 'perceptionNaturalImageTraining01'
run = 1
#task = 'trance'
#suffix = 'inplane'
datatype = 'anat'

In [54]:
bids_root = os.path.normpath(os.path.join(os.getcwd(), '..', dataset))
print(bids_root)

C:\Users\PaulDRP\source\repos\ds001506


In [55]:
if not os.path.isdir(bids_root):
    os.makedirs(bids_root)

#### 1.1) Download Data (BIDS-standard repo)

In [56]:
#openneuro.download(dataset=dataset, target_dir=bids_root)
openneuro.download(dataset=dataset, target_dir=bids_root, include=[f'sub-{subject}'])


👋 Hello! This is openneuro-py 2022.1.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds001506 …
👉 Retrieving up to 617 files (5 concurrent downloads).
✅ Finished downloading ds001506.

🧠 Please enjoy your brains.



Skipping dataset_description.json: already downloaded.: 100%|##########| 740/740 [00:00<?, ?B/s]

Skipping README: already downloaded.: 100%|##########| 6.78k/6.78k [00:00<?, ?B/s]

Skipping CHANGES: already downloaded.: 100%|##########| 397/397 [00:00<?, ?B/s]

#### 1.2) Load Data

In [87]:
#bids_path = mne_bids.BIDSPath(root=bids_root, subject=subject, session=session, datatype=datatype, task=task, suffix=suffix, extension='.set')
bids_path = mne_bids.BIDSPath(root=bids_root, subject=subject, session=session, datatype=datatype)
print(bids_path)

C:/Users/PaulDRP/source/repos/ds001506/sub-01/ses-perceptionNaturalImageTraining01/anat/sub-01_ses-perceptionNaturalImageTraining01


In [88]:
raw = mne_bids.read_raw_bids(bids_path=bids_path)

ValueError: Suffix anat is not allowed. Use one of these suffixes ['meg', 'markers', 'eeg', 'ieeg', 'T1w', 'FLASH', 'participants', 'scans', 'electrodes', 'optodes', 'channels', 'coordsystem', 'events', 'headshape', 'digitizer', 'beh', 'physio', 'stim', 'nirs'].

In [72]:
print(raw.info['subject_info'])

NameError: name 'raw' is not defined

In [ ]:
print(raw.info['sfreq'])

In [ ]:
print(raw.annotations)

In [ ]:
print(raw.annotations[0])

In [ ]:
print(raw.annotations[1])

In [ ]:
print(raw.annotations[2])

In [ ]:
raw.plot()

In [ ]:
mne_bids.inspect_dataset(bids_path)

#### 1.3) Format BIDS into Tensorflow Dataset

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])

### Preloaded Dataset

In [24]:
dat_dir = os.path.normpath(os.path.join(os.getcwd(), '..', 'DeepImageReconstruction'))

In [25]:
# GPU usage settings
decoded_features_dir = os.path.normpath(os.path.join(dat_dir, 'data', 'decodedfeatures'))

In [26]:
def decode_feature_filename(net, layer, subject, roi, image_type, image_label):
    name = '%s-%s-%s-%s-%s-%s.mat' % (image_type, net, layer, subject, roi, image_label)
    return os.path.join(decoded_features_dir, image_type, net, layer, subject, roi, name)

In [42]:
# Data settings
results_dir = os.path.normpath(os.path.join(os.getcwd(), 'results'))

In [28]:
subjects_list = ['S1', 'S2', 'S3']

rois_list = ['VC']

network = 'VGG19'

In [66]:
save_dir_root = results_dir
print(save_dir_root)
if not os.path.exists(save_dir_root):
    os.makedirs(save_dir_root)

C:\Users\PaulDRP\source\repos\eeg1\results


In [29]:
# Images in figure 2
image_type = 'natural'
image_label_list = ['Img0009', 'Img0002', 'Img0001', 'Img0005', 'Img0036', 'Img0045', 'Img0031', 'Img0043']
n_iteration = 200

In [30]:
# Average image of ImageNet
img_mean_file = os.path.normpath(os.path.join(dat_dir, 'data', 'ilsvrc_2012_mean.npy'))
img_mean = np.load(img_mean_file)
img_mean = np.float32([img_mean[0].mean(), img_mean[1].mean(), img_mean[2].mean()])

In [174]:
# CNN model
model_file = os.path.normpath(os.path.join(dat_dir, 'net', 'VGG_ILSVRC_19_layers', 'VGG_ILSVRC_19_layers.caffemodel'))
prototxt_file = os.path.normpath(os.path.join(dat_dir, 'net', 'VGG_ILSVRC_19_layers', 'VGG_ILSVRC_19_layers.prototxt'))
channel_swap = (2, 1, 0)
#net = caffe.Classifier(prototxt_file, model_file, mean=img_mean, channel_swap=channel_swap)
#h, w = net.blobs['data'].data.shape[-2:]
#net.blobs['data'].reshape(1, 3, h, w)

In [32]:
# Generator network
model_file = os.path.normpath(os.path.join(dat_dir, 'net', 'generator_for_inverting_fc7', 'generator.caffemodel'))
prototxt_file = os.path.normpath(os.path.join(dat_dir, 'net', 'generator_for_inverting_fc7', 'generator.prototxt'))
net_gen = caffe.Net(prototxt_file, model_file, caffe.TEST)
input_layer_gen = 'feat'        # Input layer for generator net
output_layer_gen = 'generated'  # Output layer for generator net

NameError: name 'caffe' is not defined

In [ ]:
# Feature size for input layer of the generator net
feat_size_gen = net_gen.blobs[input_layer_gen].data.shape[1:]
num_of_unit = net_gen.blobs[input_layer_gen].data[0].size

In [ ]:
# Upper bound for input layer of the generator net
bound_file = './data/act_range/3x/fc7.txt'
upper_bound = np.loadtxt(bound_file, delimiter=' ', usecols=np.arange(0, num_of_unit), unpack=True)
upper_bound = upper_bound.reshape(feat_size_gen)

In [ ]:
# Initial features for the input layer of the generator (we use a 0 vector as initial features)
initial_gen_feat = np.zeros_like(net_gen.blobs[input_layer_gen].data[0])

In [169]:
# Feature SD estimated from true CNN features of 10000 images
feat_std_file = os.path.normpath(os.path.join(dat_dir, 'data', 'estimated_vgg19_cnn_feat_std.mat'))
print(feat_std_file)
feat_std0 = sio.loadmat(feat_std_file)

C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\estimated_vgg19_cnn_feat_std.mat


In [91]:
print(model.layers)

[<keras.engine.input_layer.InputLayer object at 0x00000193E0A4F100>, <keras.layers.convolutional.Conv2D object at 0x00000193E0AC2650>, <keras.layers.convolutional.Conv2D object at 0x00000193E0A5C490>, <keras.layers.pooling.MaxPooling2D object at 0x00000193E0AC2920>, <keras.layers.convolutional.Conv2D object at 0x00000193E0A4F6A0>, <keras.layers.convolutional.Conv2D object at 0x00000193E0B3BA90>, <keras.layers.pooling.MaxPooling2D object at 0x00000193E1170A90>, <keras.layers.convolutional.Conv2D object at 0x00000193E0B3B460>, <keras.layers.convolutional.Conv2D object at 0x00000193E1171EA0>, <keras.layers.convolutional.Conv2D object at 0x00000193E11726B0>, <keras.layers.convolutional.Conv2D object at 0x00000193E1173970>, <keras.layers.pooling.MaxPooling2D object at 0x00000193E1170B20>, <keras.layers.convolutional.Conv2D object at 0x00000193E1172DA0>, <keras.layers.convolutional.Conv2D object at 0x00000193E117DD80>, <keras.layers.convolutional.Conv2D object at 0x00000193E117EAD0>, <keras.

In [152]:
# CNN Layers (all conv and fc layers)
#layers = [layer for layer in net.blobs.keys() if 'conv' in layer or 'fc' in layer]
layers = []

#print(tf.keras.layers.Conv2D.__name__)

count = 0
pool_count = 1
block_count = 1
for layer in model.layers:
    if layer.__class__.__name__ == 'InputLayer':
        print(f'layer {count}: InputLayer')
        #layers.append('InputLayer')
    if layer.__class__.__name__ == 'Conv2D':
        sections = layer.name.split('_')
        block = sections[0]
        blockNum = block[-1]
        if int(blockNum) > block_count:
            block_count = int(blockNum)
        layerNum = sections[1][-1]
        val = 'conv' + blockNum + '_' + layerNum
        print(f'layer {count}: {layer.name} is {val}')
        layers.append(val)
    elif layer.__class__.__name__ == 'MaxPooling2D':
        sections = layer.name.split('_')
        block = sections[0]
        blockNum = block[-1]
        if int(blockNum) > block_count:
            block_count = int(blockNum)
        val = 'pool' + blockNum
        print(f'layer {count}: {layer.name} is {val}')
        layers.append(val)
        pool_count = pool_count + 1
    elif layer.__class__.__name__ == 'Flatten':
        print(f'layer {count}: Flatten')
        #layers.append('Flatten')
    elif layer.__class__.__name__ == 'Dense':
        #print(f'layer {count}: Dense')
        block_count = block_count + 1
        val = 'fc' + str(block_count)
        print(f'layer {count}: {layer.name} is {val}')
        layers.append(val)
    else:
        print('Error finding layer')
    count = count + 1
    


layer 0: InputLayer
Error finding layer
layer 1: block1_conv1 is conv1_1
layer 2: block1_conv2 is conv1_2
layer 3: block1_pool is pool1
layer 4: block2_conv1 is conv2_1
layer 5: block2_conv2 is conv2_2
layer 6: block2_pool is pool2
layer 7: block3_conv1 is conv3_1
layer 8: block3_conv2 is conv3_2
layer 9: block3_conv3 is conv3_3
layer 10: block3_conv4 is conv3_4
layer 11: block3_pool is pool3
layer 12: block4_conv1 is conv4_1
layer 13: block4_conv2 is conv4_2
layer 14: block4_conv3 is conv4_3
layer 15: block4_conv4 is conv4_4
layer 16: block4_pool is pool4
layer 17: block5_conv1 is conv5_1
layer 18: block5_conv2 is conv5_2
layer 19: block5_conv3 is conv5_3
layer 20: block5_conv4 is conv5_4
layer 21: block5_pool is pool5
layer 22: Flatten
layer 23: fc1 is fc6
layer 24: fc2 is fc7
layer 25: predictions is fc8


In [172]:
# Set reconstruction options
opts = {
    # The loss function type: {'l2','l1','inner','gram'}
    'loss_type': 'l2',

    # The total number of iterations for gradient descend
    'iter_n': n_iteration,

    # Learning rate
    'lr_start': 2.,
    'lr_end': 1e-10,

    # Gradient with momentum
    'momentum_start': 0.9,
    'momentum_end': 0.9,

    # Decay for the features of the input layer of the generator after each
    # iteration
    'decay_start': 0.01,
    'decay_end': 0.01,

    # Name of the input layer of the generator (str)
    'input_layer_gen': 'in1', #input_layer_gen,

    # Name of the output layer of the generator (str)
    'output_layer_gen': 'out1', # output_layer_gen,

    # Upper and lower boundary for the input layer of the generator
    'feat_upper_bound': upper_bound,
    'feat_lower_bound': 0.,

    # The initial features of the input layer of the generator (setting to
    # None will use random noise as initial features)
    'initial_gen_feat': initial_gen_feat,

    # Display the information on the terminal for every n iterations
    'disp_every': 1
}

NameError: name 'upper_bound' is not defined

In [162]:
# Save the optional parameters
with open(os.path.join(save_dir_root, 'options.pkl'), 'w') as f:
    pickle.dump(opts, f)

NameError: name 'opts' is not defined

In [165]:
def estimate_cnn_feat_std(cnn_feat):
    '''
    estimate the std of the CNN features

    INPUT:
        cnn_feat: CNN feature array [channel,dim1,dim2] or [1,channel];

    OUTPUT:
        cnn_feat_std: std of the CNN feature,
        here the std of each channel is estimated first,
        then average std across channels;
    '''
    feat_ndim = cnn_feat.ndim
    feat_size = cnn_feat.shape
    # for the case of fc layers
    if feat_ndim == 1 or (feat_ndim == 2 and feat_size[0] == 1) or (feat_ndim == 3 and feat_size[1] == 1 and feat_size[2] == 1):
        cnn_feat_std = np.std(cnn_feat)
    # for the case of conv layers
    elif feat_ndim == 3 and (feat_size[1] > 1 or feat_size[2] > 1):
        num_of_ch = feat_size[0]
        # std for each channel
        cnn_feat_std = np.zeros(num_of_ch, dtype='float32')
        for j in range(num_of_ch):
            feat_ch = cnn_feat[j, :, :]
            cnn_feat_std[j] = np.std(feat_ch)
        cnn_feat_std = np.mean(cnn_feat_std)  # std averaged across channels
    return cnn_feat_std

In [170]:
# Reconstrucion

for subject, roi, image_label in product(subjects_list, rois_list, image_label_list):

    print('')
    print('Subject:     ' + subject)
    print('ROI:         ' + roi)
    print('Image label: ' + image_label)
    print('')

    save_dir = os.path.join(save_dir_root, subject, roi)
    print(f'save dir: {save_dir}')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Load the decoded CNN features
    features = {}
    print('loading decoded CNN features')
    for layer in layers:
        # The file full name depends on the data structure for decoded CNN features
        file_name = decode_feature_filename(network, layer, subject, roi, image_type, image_label)
        print(file_name)
        feat = sio.loadmat(file_name)['feat']
        if 'fc' in layer:
            feat = feat.reshape(feat.size)

        # Correct the norm of the decoded CNN features
        feat_std = estimate_cnn_feat_std(feat)
        feat = (feat / feat_std) * feat_std0[layer]

        features.update({layer: feat})

    # Weight of each layer in the total loss function

    # Norm of the CNN features for each layer
    feat_norm = np.array([np.linalg.norm(features[layer]) for layer in layers], dtype='float32')

    # Use the inverse of the squared norm of the CNN features as the weight for each layer
    weights = 1. / (feat_norm ** 2)

    # Normalise the weights such that the sum of the weights = 1
    weights = weights / weights.sum()
    layer_weight = dict(zip(layers, weights))

    opts.update({'layer_weight': layer_weight})

    # Reconstruction
    snapshots_dir = os.path.join(save_dir, 'snapshots', 'image-%s' % image_label)
    recon_img, loss_list = reconstruct_image(features, net, net_gen,
                                             save_intermediate=True,
                                             save_intermediate_path=snapshots_dir,
                                             **opts)

    # Save the results

    # Save the raw reconstructed image
    save_name = 'recon_img' + '-' + image_label + '.mat'
    sio.savemat(os.path.join(save_dir, save_name), {'recon_img': recon_img})

    # To better display the image, clip pixels with extreme values (0.02% of
    # pixels with extreme low values and 0.02% of the pixels with extreme high
    # values). And then normalise the image by mapping the pixel value to be
    # within [0,255].
    save_name = 'recon_img_normalized' + '-' + image_label + '.jpg'
    PIL.Image.fromarray(normalise_img(clip_extreme_value(recon_img, pct=4))).save(os.path.join(save_dir, save_name))



Subject:     S1
ROI:         VC
Image label: Img0009

save dir: C:\Users\PaulDRP\source\repos\eeg1\results\S1\VC
loading decoded CNN features
C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\decodedfeatures\natural\VGG19\conv1_1\S1\VC\natural-VGG19-conv1_1-S1-VC-Img0009.mat
C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\decodedfeatures\natural\VGG19\conv1_2\S1\VC\natural-VGG19-conv1_2-S1-VC-Img0009.mat
C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\decodedfeatures\natural\VGG19\pool1\S1\VC\natural-VGG19-pool1-S1-VC-Img0009.mat
C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\decodedfeatures\natural\VGG19\conv2_1\S1\VC\natural-VGG19-conv2_1-S1-VC-Img0009.mat
C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\decodedfeatures\natural\VGG19\conv2_2\S1\VC\natural-VGG19-conv2_2-S1-VC-Img0009.mat
C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\decodedfeatures\natural\VGG19\pool2\S1\VC\natural-VGG19-pool2-S1-VC-Img0009.mat
C:\User

NameError: name 'opts' is not defined

### Build model

In [ ]:
model = tf.keras.applications.vgg19.VGG19()
#model = tf.keras.Sequential()

In [59]:
model.compile()
#model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=tf.keras.optimizers.Adam(), metrics=tf.keras.metrics.Accuracy())

Re-downloading sub-01_ses-perceptionNaturalImageTraining04_task-perception_run-03_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining04_task-perception_run-04_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining05_inplaneT2.nii.gz: file size mismatch.: 0.00B [00:00…

Re-downloading sub-01_ses-perceptionNaturalImageTraining04_task-perception_run-05_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining04_task-perception_run-06_bold.nii.gz: file size misma…

...

Re-downloading sub-01_ses-perceptionNaturalImageTraining11_task-perception_run-07_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining11_task-perception_run-08_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining12_task-perception_run-02_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining12_task-perception_run-01_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining12_task-perception_run-03_bold.nii.gz: file size misma…

In [60]:
model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

Re-downloading sub-01_ses-perceptionNaturalImageTraining12_task-perception_run-04_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining13_inplaneT2.nii.gz: file size mismatch.: 0.00B [00:00…

Re-downloading sub-01_ses-perceptionNaturalImageTraining12_task-perception_run-05_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining12_task-perception_run-06_bold.nii.gz: file size misma…

...

Re-downloading sub-01_ses-perceptionNaturalImageTraining15_task-perception_run-03_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining15_task-perception_run-05_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining15_task-perception_run-06_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining15_task-perception_run-08_bold.nii.gz: file size misma…

Re-downloading sub-01_ses-perceptionNaturalImageTraining15_task-perception_run-07_bold.nii.gz: file size misma…

#### Train

In [ ]:
#model.fit(, epochs=10)

#### Evaluate

In [ ]:
#loss, accuracy = model.evaluate(, verbose=2)